# Вводная информация

В этом проекте предполагается работа с датасетом из открытого источника.  
Потребуется разработать модель машинного обучения, а также подготовить библиотеку и интерфейс к ней для предсказания на тестовой выборке.

В ходе работы над проектом будут решены следующие задачи:  
1) Исследование датасета  
2) Предобработка данных  
3) Обучение модели  
4) Подготовка предсказания на тестовой выборке  
5) Подготовка скриптов и библиотеки для обработки данных и предсказания на тестовой выборке  
6) Написание инструмента для тестирования  
7) Оформление документации  

Данные пациентов для предсказания риска сердечных приступов:  
1) id  - id  
2) Антропометрические параметры (вес, возраст, рост)  
3) Привычки (курение, качество сна и т.д)  
4) Давление  
5) Наличие хронических заболеваний  
6) Биохимия крови  
7) Таргет - высокий или низкий риск поражения сердца  


<b><i>Что имеем в таблице</i></b>  
<b>Высокая ценность для модели</b>:  
Age (возраст) – количество полных лет.  
Cholesterol (холестерин) – уровень общего холестерина в крови (ммоль/л или мг/дл).  
Heart rate (частота сердечных сокращений) – пульс в состоянии покоя (ударов в минуту).  
Diabetes (диабет) – наличие диагноза диабета (0 – нет, 1 – есть).  
Family History (наследственность) – наличие сердечно-сосудистых заболеваний у ближайших родственников (0/1).  
Smoking (курение) – факт курения (0 – не курит, 1 – курит).  
Obesity (ожирение) – наличие ожирения (0/1).  
Previous Heart Problems (анамнез сердечных проблем) – наличие ранее диагностированных сердечных заболеваний (0/1).  
Medication Use (приём лекарств) – факт приёма сердечно-сосудистых или сопутствующих препаратов (0/1).  
BMI (индекс массы тела) – показатель веса относительно роста.  
Triglycerides (триглицериды) – уровень триглицеридов в крови.  
Blood sugar (уровень сахара в крови) – показатель глюкозы (ммоль/л или мг/дл).  
Systolic blood pressure (систолическое давление) – верхнее значение артериального давления (мм рт. ст.).  
Diastolic blood pressure (диастолическое давление) – нижнее значение артериального давления (мм рт. ст.).  

<b>Средняя ценность для модели</b>:
Alcohol Consumption (употребление алкоголя) – частота или факт употребления алкоголя (0/1 или категориально).  
Exercise Hours Per Week (часы тренировок в неделю) – среднее количество часов физической активности за неделю.  
Diet (диета) – качество или тип питания (может быть категориально: сбалансированная/несбалансированная и т.д.).  
Stress Level (уровень стресса) – субъективная или измеренная оценка уровня стресса (шкала).  
Sedentary Hours Per Day (сидячие часы в день) – количество часов в день, проводимых в сидячем положении.  
Physical Activity Days Per Week (дни активности в неделю) – количество дней с умеренной или высокой физической активностью.  
Sleep Hours Per Day (часы сна в день) – среднее количество часов сна за сутки.  
Gender (пол) – 0 – женский, 1 – мужской (или наоборот).  

<b>Потенциально бесполезные признаки</b>:  
Income (доход) – уровень дохода, будет корелировать с образом жизни.  

<b>Признаки указывающие УЖЕ на проблемы с сердцем и могу дать утечку данных</b>:  
CK-MB – уровень изофермента креатинфосфокиназы MB, маркера повреждения сердца.  
Troponin (тропонин) – уровень тропонина, маркер повреждения миокарда.  

<b>Технические признаки</b>:  
id – идентификатор записи.  
Heart Attack Risk (Binary) (риск инфаркта, бинарно) – целевой признак: 0 – нет риска, 1 – есть риск.  

# Ипорты

In [ ]:
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd
import seaborn as sns
import numpy as np
import os
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Классы и функции проекта

In [ ]:
# Чистит названия столбцов, преобразует к общепринятому виду. В строках удаляет лишние пробелы, если есть
class ColumnNameProcessor:
    @staticmethod
    def process_spaces(s):
        """Удаляет лишние пробелы внутри строки и по краям."""
        if isinstance(s, str):
            s = s.strip()
            s = ' '.join(s.split())
        return s

    @staticmethod
    def replace_spaces(s):
        """Заменяет пробелы на нижнее подчеркивание в строках."""
        if isinstance(s, str):
            s = s.strip()
            s = '_'.join(s.split())
        return s

    def clean_dataframe(self, df):
        """
        Приводит названия колонок к нижнему регистру,
        убирает пробелы, заменяет их на '_',
        а также чистит строки внутри DataFrame.
        """
        # Названия колонок
        df.columns = [self.replace_spaces(self.process_spaces(col)).lower() for col in df.columns]
        
        # Чистка строковых значений
        df = df.map(self.process_spaces)
        df = df.map(lambda x: self.process_spaces(x).lower() if isinstance(x, str) else x)
        
        return df
    
class DataPreprocessor:
    """
    Класс для предобработки данных и сохранения результатов в CSV и Pickle.

    Шаги:
        1. Установка 'id' в качестве индекса.
        2. Удаление служебных столбцов.
        3. Очистка названий столбцов.
        4. В строках с пропусками заменить на -1, что эквивалетно "Unknown".
        5. Фильтрация 'gender' (male/female), изменить 1 на male, 0 на female.
        6. Удаление заданных признаков.
        7. One-Hot кодирование категориальных признаков.
        8. Удаление дубликатов.
        9. Сохранение CSV и Pickle.
    """

    def __init__(
        self,
        drop_cols=None,
        ohe_cols=None,
        processed_dir="../data/processed/",
        models_dir="../models/"
    ):
        """
        Args:
            drop_cols (list): признаки для удаления.
            ohe_cols (list): признаки для One-Hot кодирования.
            processed_dir (str): папка для train_data.csv.
            models_dir (str): папка для preprocessor.pkl.
        """
        self.drop_cols = drop_cols or []
        self.ohe_cols = ohe_cols or []
        self.processed_dir = processed_dir
        self.models_dir = models_dir
        self.encoder = None

        os.makedirs(self.processed_dir, exist_ok=True)
        os.makedirs(self.models_dir, exist_ok=True)

    def fit_transform(self, df: pd.DataFrame):
        """
        Обучает препроцессор и применяет преобразования к DataFrame.

        Args:
            df (pd.DataFrame): исходный DataFrame.

        Returns:
            pd.DataFrame: обработанный DataFrame.
        """
        # 1. ID в индекс
        df = df.set_index('id', drop=True)

        # 2. Удаление служебного столбца
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])

        # 3. Очистка названий столбцов
        df = ColumnNameProcessor().clean_dataframe(df)

        # 4. Удаление строк с пропусками
        df = df.dropna()

        # 5. Фильтрация 'gender'
        df["gender"] = df["gender"].replace({1.0: "male", 0.0: "female"})

        # 6. Удаление признаков
        df = df.drop(columns=self.drop_cols, errors="ignore")

        # 7. One-Hot кодирование
        self.encoder = OneHotEncoder(sparse_output=False, drop='first')
        encoded = self.encoder.fit_transform(df[self.ohe_cols])
        encoded_df = pd.DataFrame(
            encoded,
            columns=self.encoder.get_feature_names_out(self.ohe_cols),
            index=df.index
        )
        df = pd.concat([df.drop(columns=self.ohe_cols), encoded_df], axis=1)

        # 8. Удаление дубликатов
        df = df.drop_duplicates()

        # 9. Сохранение
        df.to_csv(os.path.join(self.processed_dir, "train_data.csv"))
        with open(os.path.join(self.models_dir, "preprocessor.pkl"), "wb") as f:
            pickle.dump(self, f)

        return df
    
def plot_combined(data, col=None, target=None, col_type=None, legend_loc='best'):
    """
    Строит графики для числовых столбцов в DataFrame, автоматически определяя их типы (дискретные или непрерывные).

    :param data: DataFrame, содержащий данные для визуализации.
    :param col: Список столбцов для построения графиков. Если None, будут использованы все числовые столбцы.
    :param target: Столбец, по которому будет производиться разделение (для hue в графиках).
    :param col_type: Словарь, определяющий типы столбцов ('col' для непрерывных и 'dis' для дискретных).
                     Если None, типы будут определены автоматически.
    :param legend_loc: Положение легенды для графиков (по умолчанию 'best').
    :return: None. Графики отображаются с помощью plt.show().
    """
    
    # Определяем числовые столбцы
    if col is None:
        numerical_columns = data.select_dtypes(include=['int', 'float']).columns.tolist()
    else:
        numerical_columns = col

    # Если col_type не указан, определяем типы автоматически
    if col_type is None:
        col_type = {}
        for col in numerical_columns:
            unique_count = data[col].nunique()
            if unique_count > 20:
                col_type[col] = 'col'  # Непрерывные данные
            else:
                col_type[col] = 'dis'  # Дискретные данные

    total_plots = len(numerical_columns) * 2
    ncols = 2
    nrows = (total_plots + ncols - 1) // ncols

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 5 * nrows))
    axs = axs.flatten()

    index = 0

    for col in numerical_columns:
        # Определяем тип графика
        plot_type = col_type.get(col)
        if plot_type is None:
            raise ValueError(f"Тип для столбца '{col}' не указан в col_type.")

        # Гистограмма или countplot
        if index < len(axs):
            if plot_type == 'col':
                if target is not None:
                    sns.histplot(data, x=col, hue=target, bins=20, kde=True, ax=axs[index])
                    handles, labels = axs[index].get_legend_handles_labels()
                    if handles:
                        axs[index].legend(title=target, loc=legend_loc)
                else:
                    sns.histplot(data[col].dropna(), bins=20, kde=True, ax=axs[index])
                axs[index].set_title(f'Гистограмма: {col}')
                axs[index].tick_params(axis='x', rotation=90)  # Вертикальные метки

            elif plot_type == 'dis':
                if target is not None:
                    sns.countplot(data=data, x=col, hue=target, ax=axs[index])
                    handles, labels = axs[index].get_legend_handles_labels()
                    if handles:
                        axs[index].legend(title=target, loc=legend_loc)
                else:
                    sns.countplot(data=data, x=col, ax=axs[index])
                axs[index].set_title(f'Countplot: {col}')
                axs[index].tick_params(axis='x', rotation=90)  # Вертикальные метки

            index += 1

        # Боксплот
        if index < len(axs):
            sns.boxplot(x=data[col], ax=axs[index])
            axs[index].set_title(f'Боксплот: {col}')
            index += 1

    # Отключаем оставшиеся оси
    for j in range(index, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()
    
def plot_categorical_columns(data, col=None, target=None):
    """
    Функция для визуализации категориальных данных с возможностью группировки по целевому столбцу.
    
    :param data: DataFrame с категориальными данными
    :param col: Название столбца для визуализации (по умолчанию None — визуализируются все категориальные столбцы)
    :param target: Название столбца для группировки данных (по умолчанию None — без группировки)
    :return: None — функция отображает графики
    """
    categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
    
    if col is not None and col in categorical_columns:
        categorical_columns = [col]
    elif col is not None:
        print(f"Столбец '{col}' не найден в данных.")
        return

    n = len(categorical_columns)
    ncols = 2
    nrows = (n * 2 + ncols - 1) // ncols

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 6 * nrows))
    axs = axs.flatten()

    index = 0
    colors = plt.cm.tab10.colors

    for col in categorical_columns:
        unique_values = data[col].value_counts().index
        color_map = {value: colors[i] for i, value in enumerate(unique_values)}

        # Визуализация круговой диаграммы
        grouped_data = data[col].value_counts()
        axs[index].pie(grouped_data, labels=grouped_data.index, autopct='%1.1f%%', startangle=90, colors=[color_map[val] for val in grouped_data.index])
        axs[index].set_title(f'{col} (общая)')
        axs[index].set_ylabel('')
        index += 1

        # Визуализация гистограммы
        if target is not None and target in data.columns:
            # Создаем MultiIndex для unstack
            grouped_data = data.groupby([target, col]).size().unstack(fill_value=0)
            grouped_data.plot(kind='bar', ax=axs[index], color=[color_map[val] for val in grouped_data.columns])
        else:
            data[col].value_counts().plot(kind='bar', ax=axs[index], color=[color_map[val] for val in data[col].value_counts().index])

        axs[index].set_title(f'{col} (гистограмма)')
        axs[index].set_ylabel('Частота')
        index += 1

    for j in range(index, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()
    
# пропуски
def missing_data(data):
    missing_data = data.isna().sum()
    missing_data = missing_data[missing_data > 0]
    display(missing_data)
    
def calc_target_correlations(df, target_col: str = None, drop_cols: list = None):
    """
    Считает корреляции признаков с таргетом, строит heatmap и рассчитывает VIF.
    Результаты выводятся прямо в Jupyter.
    """
    if drop_cols is None:
        drop_cols = []
    
    df_tmp = df.copy()

    # Преобразуем категориальные в числовые
    cat_cols = df_tmp.select_dtypes(include=["object", "category"]).columns
    for c in cat_cols:
        df_tmp[c] = df_tmp[c].astype("category").cat.codes

    # Числовые колонки
    numeric_cols = df_tmp.select_dtypes(exclude=["object", "category"]).columns.tolist()
    if target_col not in numeric_cols:
        raise ValueError(f"target_col '{target_col}' должен быть числовым")

    # Корреляции с target
    corr_df = (
        df_tmp[numeric_cols]
        .corr()[target_col]
        .drop(target_col)
        .sort_values(key=np.abs, ascending=False)
    )
    display("=== Корреляция с таргетом ===")
    display(corr_df)

    # Heatmap
    heatmap_cols = [col for col in numeric_cols if col not in drop_cols or col == target_col]
    corr_matrix = df_tmp[heatmap_cols].corr()

    plt.figure(figsize=(12, 10))
    plt.imshow(corr_matrix, interpolation="nearest", cmap="coolwarm", aspect="auto")
    plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90, fontsize=8)
    plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns, fontsize=8)
    plt.colorbar()
    plt.title("Correlation Heatmap (включая target)")

    for i in range(corr_matrix.shape[0]):
        for j in range(corr_matrix.shape[1]):
            value = corr_matrix.iloc[i, j]
            plt.text(j, i, f"{value:.2f}", ha="center", va="center", fontsize=6, color="black")

    plt.tight_layout()
    plt.show()

    # VIF
    vif_cols = [col for col in numeric_cols if col != target_col and col not in drop_cols]
    X_vif = df_tmp[vif_cols].copy()
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_vif), columns=vif_cols)

    vif_data = pd.DataFrame()
    vif_data["feature"] = vif_cols
    vif_data["VIF"] = [
        variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])
    ]
    vif_data = vif_data.sort_values("VIF", ascending=False)

    display("\n=== VIF ===")
    display(vif_data)
    

# Исследование датасета

## Подгружаем базы

In [ ]:
train_data = pd.read_csv('../data/raw/heart_train.csv')

## Общая статистика

In [ ]:
display(train_data.info())

In [ ]:
cols_by_dtype = {dtype: train_data.select_dtypes(include=[dtype]).columns.tolist()
                 for dtype in train_data.dtypes.unique()}

for dtype, cols in cols_by_dtype.items():
    print(dtype, ":", cols)

In [ ]:
train_data.describe().T

## Пропуски

In [ ]:
if train_data.isnull().sum().sum() > 0:
    msno.bar(train_data)
    plt.show()
    
    missing_data(train_data)

пропуски в части важных для модели признаков и части в бесполезных.

## Распределение числовых данных

In [ ]:
plot_combined(train_data, col=None, target=None, col_type=None, legend_loc='best')

CK-MB, Troponin - ненормальное распределение + мы их относим в опасным признакам и не будем использовать в обучении  
К остальным вопросов нет

## Распределение категориальных данных

In [ ]:
plot_categorical_columns(train_data, col=None, target="Heart Attack Risk (Binary)")

1 к 3, лучше бы было, если женщин/мужчин было бы поровну

In [ ]:
train_data["Gender"].value_counts()

можно будет попробовать выровнять число мужчин/женщин, по идее даст более качественную модель, посмотрим по ходу дела...  
и учитывая равное число пропусков и часть гендера в виде 0/1 можно смело заявлять, чтобы было кривое объединение 2х баз данных, но, к сожалению, оно не дало пользы, т.к. мы имеем пропуски и не знаем какой гендер под 0/1, а использовать это наугад смертельно непозволительно для нас

In [ ]:
train_data.sample(10)

## Выводы/план предобработки

Судя по примерам в выводе - данные уже почти все нормализованны.  
Что надо сделать:  
1) Удалить непонятный столбец, дублирующий номер строки;  
2) Названия столбцов привести к строчным буквам, пробелы заменить на нижнее подчеркивание;  
3) Заменить пропуски на -1, что будет считать как Unknown - сердце это не шутки, мы не можем себе позволить заполнять пропуски средними/медианами, но и удалять выборку тоже не стоит;  
4) Закодировать "редкие" признаки в OneHotEncoder - diabetes, family_history, smoking, obesity, alcohol_consumption, diet, previous_heart_problems, medication_use, gender, предварительно переведем все в int;  
5) *Обобщить признаки, сделать "lifestyle_bad" (образ жизни негативный), где перемножим Smoking (курение), Obesity (ожирение), Alcohol Consumption (употребление алкоголя), Sedentary Hours Per Day (сидячие часы в день);  
6) *Обобщить признаки, сделать "lifestyle_good" (образ жизни спортивный), где перемножим Exercise Hours Per Week (часы тренировок в неделю), Diet (диета), Physical Activity Days Per Week (дни активности в неделю);  
7) Удалить дубликаты;  

/* в потоке мыслей, проверить как поведет себя модель

# Предобработка данных

Вначале посмотрел/сделал по отдельности, потом сделал с сохранением препроцессора для последующего использования в проде

In [ ]:
drop_cols = ['income', 'ck-mb', 'troponin']
ohe_cols = [
    'diabetes', 'family_history', 'smoking', 'obesity',
    'alcohol_consumption', 'diet', 'previous_heart_problems',
    'medication_use', 'gender'
]

In [ ]:
preprocessor = DataPreprocessor(drop_cols=drop_cols, ohe_cols=ohe_cols)
train_data_processed = preprocessor.fit_transform(train_data)

In [ ]:
# и посмотреть на зависимости признаков
corr = calc_target_correlations(train_data_processed, target_col="heart_attack_risk_(binary)", )

Сильной кореляции между таргетом и признаками не наблюдается  
В целом, по базе, есть явная зависимость - взрослые мужчины курят  
Мультиколлинеарность также не наблюдается